In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [4]:
df = pd.read_csv("/content/drive/MyDrive/BGS/biogasdf_ready.csv", parse_dates=['date'])
df['Month'] = df['date'].dt.month
df['Hour'] = pd.to_datetime(df['hour'], format='%H:%M:%S').dt.hour

In [5]:
bio1_phase0 = df[(df['bio_id'] == 1) & (df['phase_test'] == 0)].reset_index()
bio1_phase1 = df[(df['bio_id'] == 1) & (df['phase_test'] == 1)].reset_index()
bio1_phase2 = df[(df['bio_id'] == 1) & (df['phase_test'] == 2)]
bio1_phase3 = df[(df['bio_id'] == 1) & (df['phase_test'] == 3)]
bio2_phase0 = df[(df['bio_id'] == 2) & (df['phase_test'] == 0)]
bio2_phase1 = df[(df['bio_id'] == 2) & (df['phase_test'] == 1)]
bio2_phase2 = df[(df['bio_id'] == 2) & (df['phase_test'] == 2)]
bio2_phase3 = df[(df['bio_id'] == 2) & (df['phase_test'] == 3)]

In [6]:
bio1_phase1.tail()

,index,date,hour,fluid_temp,ground_temp,air_umidity,air_temp,gas_umidity,gas_temp,dm3_gas,bio_id,phase_test,Month,Hour
10870,24551,2021-11-30,15:50:51,21.12,21.31,20.5,40.1,82.22,44.95,0.0,1,1,11,15
10871,24552,2021-11-30,16:05:51,21.12,21.31,14.3,49.6,79.36,43.28,0.0,1,1,11,16
10872,24553,2021-11-30,16:20:49,21.12,21.31,13.9,49.9,76.64,41.91,0.0,1,1,11,16
10873,24554,2021-11-30,16:35:49,21.12,21.31,13.7,50.4,79.63,36.26,0.0,1,1,11,16
10874,24555,2021-11-30,16:50:50,21.12,21.31,18.0,41.8,82.91,33.18,0.0,1,1,11,16


In [7]:
data = bio1_phase1.sort_values(by=['date','hour']).reset_index()

In [8]:
data.tail()

,level_0,index,date,hour,fluid_temp,ground_temp,air_umidity,air_temp,gas_umidity,gas_temp,dm3_gas,bio_id,phase_test,Month,Hour
10870,5423,13899,2022-02-28,22:46:13,23.62,24.25,85.3,23.5,84.87,21.89,0.0,1,1,2,22
10871,5424,13900,2022-02-28,23:01:12,23.62,24.25,86.4,23.3,84.81,21.66,0.0,1,1,2,23
10872,5425,13901,2022-02-28,23:16:13,23.62,24.25,87.6,23.2,84.99,21.64,0.0,1,1,2,23
10873,5426,13902,2022-02-28,23:31:10,23.62,24.25,87.8,23.2,85.09,21.65,0.0,1,1,2,23
10874,5427,13903,2022-02-28,23:46:11,23.62,24.31,88.6,23.1,84.97,21.46,0.0,1,1,2,23


In [42]:
data.shape

(10875, 15)

In [9]:
cols = list(data)[4:11]

In [10]:
dataset = data[cols].astype(float)

In [11]:
dataset.head()

,fluid_temp,ground_temp,air_umidity,air_temp,gas_umidity,gas_temp,dm3_gas
0,18.25,18.94,99.9,16.0,85.00,15.74,0.0
1,18.19,18.94,99.9,15.8,84.87,15.63,0.0
2,18.06,18.94,99.9,15.6,84.77,15.37,0.0
3,18.06,18.94,99.9,15.5,84.78,15.29,0.0
4,17.94,18.94,99.9,15.3,84.80,15.15,0.0


In [12]:
scaler = StandardScaler()
dataset_scaled = scaler.fit_transform(dataset)

In [13]:
trainX = []
trainy = []

In [14]:
n_future = 1
n_past = 24

In [15]:
for i in range(n_past, len(dataset_scaled)-n_future+1):
    trainX.append(dataset_scaled[i-n_past:i, 0:dataset_scaled.shape[1]])
    trainy.append(dataset_scaled[i+n_future - 1: i + n_future, 0])

trainX, trainy = np.array(trainX), np.array(trainy)

In [19]:
format(trainX.shape)

'(10851, 24, 7)'

In [20]:
format(trainy.shape)

'(10851, 1)'

In [30]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainy.shape[1]))

In [31]:
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 24, 64)            18432     
                                                                 
 lstm_4 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 30,881
Trainable params: 30,881
Non-trainable params: 0
_________________________________________________________________


In [34]:
history = model.fit(trainX, trainy, epochs=10, verbose=14, validation_split=0.1, batch_size=40)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [52]:
n_future = 24
forecast_data = pd.date_range(list(data.date)[-1], periods=n_future, freq='1d').to_list
forecast = model.predict(trainX[-n_future:])

1/1 [==============================] - 0s 20ms/step


In [54]:
#forecast